In [1]:
import requests
import csv
from datetime import datetime

keys, transactions = None, []
error = []
current_time = 1654016400
last_transactions = set()

while True:
    query = """
    {
        userTransactions(where: {timestamp_lte: """ + str(current_time) + """}, first: 1000, orderBy: timestamp, orderDirection: desc) {
            id
            pool {
                id
            }
            user {
                id
            }
            timestamp
        }
    }
    """
    response = requests.post('https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
                                '',
                                json={'query': query})
    if response.status_code != 200:
        print("Problem reading from timestamp", current_time, ":", response.status_code)
        continue
    
    try:
        data = response.json()["data"]["userTransactions"]
    except Exception:
        print("Error at timestamp", current_time)
        continue

    if len(data) == 0:
        break
    
    if keys is None:
        keys = data[0].keys()

    index = 0
    try:
        while data[index]["id"] in last_transactions:
            index += 1
    except IndexError:
            current_time -= 1
            continue
    print(len(data) - index, "transactions found at timestamp", current_time)

    for transaction in data[index:]:
        transaction["pool"] = transaction["pool"]["id"]
        transaction["user"] = transaction["user"]["id"]
        transactions.append(transaction)

    current_time = int(data[-1]["timestamp"])
    index = -1
    last_transactions = set()
    while data[index]["timestamp"] == data[-1]["timestamp"]:
        last_transactions.add(data[index]["id"])
        index -= 1


with open('allTransaction.csv', 'w', newline='') as output_file:
    DICT_WRITER = csv.DictWriter(output_file, keys)
    DICT_WRITER.writeheader()
    DICT_WRITER.writerows(transactions)

1000 transactions found at timestamp 1654016400
996 transactions found at timestamp 1653946304
997 transactions found at timestamp 1653889319
998 transactions found at timestamp 1653823029
998 transactions found at timestamp 1653779219
997 transactions found at timestamp 1653725539
994 transactions found at timestamp 1653678449
999 transactions found at timestamp 1653631728
996 transactions found at timestamp 1653595823
999 transactions found at timestamp 1653563109
999 transactions found at timestamp 1653499266
996 transactions found at timestamp 1653442892
996 transactions found at timestamp 1653375569
998 transactions found at timestamp 1653314456
997 transactions found at timestamp 1653251359
999 transactions found at timestamp 1653189464
999 transactions found at timestamp 1653129132
999 transactions found at timestamp 1653075012
999 transactions found at timestamp 1653021230
998 transactions found at timestamp 1652969383
998 transactions found at timestamp 1652932934
997 transact

In [2]:
!wc -l allTransaction.csv
# 1145565

1165538 allTransaction.csv


In [3]:
from google.colab import files
files.download('allTransaction.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
!ls -la

total 210756
drwxr-xr-x 1 root root      4096 Jun  1 14:28 .
drwxr-xr-x 1 root root      4096 Jun  1 14:14 ..
-rw-r--r-- 1 root root 215795017 Jun  1 14:28 allTransaction.csv
drwxr-xr-x 4 root root      4096 May 17 13:38 .config
drwxr-xr-x 1 root root      4096 May 17 13:39 sample_data
